In [1]:
# install pydp to the local conda environment

!cd ../PyDP && poetry install && cd ../pyspark_dp_beta

Installing dependencies from lock file

Package operations: 0 installs, 76 updates, 0 removals

  • Updating decorator (5.0.5 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 5.0.9): Pending...
  • Updating ipython-genutils (0.2.0 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 0.2.0): Pending...
  • Updating six (1.15.0 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 1.16.0): Pending...
  • Updating typing-extensions (3.7.4.3 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 3.10.0.0): Pending...
  • Updating zipp (3.4.1 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 3.4.1): Pending...
  • Updating typing-extensions (3.7.4.3 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 3.10.0.0): Pending...
  • Updating zipp (3.4.1 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 3.4.1): Pending...
  • Updating six (1.15.0 /usr/lib/anaconda-wmf/lib/python3.7/site-packages -> 1.16.0): Installing...
  • Updating typing-extensions (3.7.4.3 /usr/lib/anaconda-wmf/lib/pyth

In [2]:
import pydp
import wmfdata
from dataclasses import dataclass
import numpy as np
import random
import math
from IPython.display import Latex

In [3]:
spark = wmfdata.spark.get_session(
    app_name='pyspark-large; differential privacy; htriedman',
    type='yarn-large',
    ship_python_env=True
)

A conda environment is already packed at conda-2021-08-31T19.46.22_htriedman.tgz. If you have recently installed new packages into your conda env, set force=True in conda_pack_kwargs and it will be repacked for you.
Will ship conda-2021-08-31T19.46.22_htriedman.tgz to remote Spark executors.
PySpark executors will use conda-2021-08-31T19.46.22_htriedman/bin/python3.


In [4]:
# get (page title, page id, project, country, actor signature) for Aug 15 2021 UTC6:00

rdd = spark.sql("""
SELECT
  pageview_info['page_title'] as page_title,
  page_id,
  pageview_info['project'] as project,
  geocoded_data['country'] as country,
  actor_signature
FROM wmf.pageview_actor
WHERE year = 2021 AND month = 8 AND day = 15 AND hour = 6 AND page_id IS NOT NULL
""").rdd

In [5]:
# add laplace noise to a single number
def add_laplace_noise(x, eps, sensitivity):
    return x + pydp.distributions.LaplaceDistribution(eps, sensitivity).sample()

# add laplace noise to a spark rdd
def add_laplace_noise_to_rdd(rdd, eps, max_partitions, max_per_partition):
    eps_per_partition = eps / max_partitions
    sensitivity_per_partition = max_per_partition
    return rdd.map(lambda x: (x[0], add_laplace_noise(x[1], eps_per_partition, sensitivity_per_partition)))

In [6]:
# add gaussian noise to a single number
def add_gaussian_noise(x, eps, delta, sensitivity):
    sigma_squared = (2 * math.log(1.25 / delta) * sensitivity**2) / (eps**2)
    return x + pydp.distributions.GaussianDistribution(sigma_squared).sample()

# add laplace noise to a spark rdd
def add_gaussian_noise_to_rdd(rdd, eps, delta, max_partitions, max_per_partition):
    eps_per_partition = eps / max_partitions
    sensitivity_per_partition = max_per_partition
    return rdd.map(lambda x: (x[0], add_gaussian_noise(x[1], eps_per_partition, delta, sensitivity_per_partition)))

In [7]:
def calculate_threshold(eps, delta, max_partitions, max_per_partition):
    eps_per_partition = eps / max_partitions
    sensitivity_per_partition = max_per_partition
    b = sensitivity_per_partition / eps_per_partition
    return -b * math.log(2 * b * delta)

In [8]:
# do bounded DP count
def do_count(rdd, eps, delta, max_partitions, max_per_partition, noise_kind):
    # rekey to a tuple of (actor signature, page id)
    # ((actor_signature, page_id), pageview)
    dp_count_rdd = rdd.map(lambda x: ((x.actor_signature, x.page_id), [x]))

    # randomly get a set of at most `max_per_partition` pageviews for each (actor signature, page id) tuple
    # ((actor_signature, page_id), [pageview]) {max length of max_per_partition}
    dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_per_partition)))

    # rekey to just actor signature
    # (actor_signature, [pageview]) {with redundancies}
    dp_count_rdd = dp_count_rdd.map(lambda x: ((x[0][0], x[1])))

    # randomly get a set of at most `max_partitions` sets of pageviews for each actor signature
    # (actor_signature, [pageview]) {max length of max_per_partition * max_partitions}
    dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_partitions)))

    # drop actor signature as key
    # ([pageview])
    dp_count_rdd = dp_count_rdd.map(lambda x: x[1])

    # unnest lists of pageviews using a flatmap
    # (pageview)
    dp_count_rdd = dp_count_rdd.flatMap(lambda x: x)

    # now that contributions are bounded, count views per tuple
    dp_count_rdd = dp_count_rdd.map(lambda x: ((x.project, x.country, x.page_id, x.page_title), 1))
    dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: (x + y))

    if noise_kind == "laplace":
        # add laplace noise to counts
        dp_count_rdd = add_laplace_noise_to_rdd(dp_count_rdd, eps, max_partitions, max_per_partition)
    elif noise_kind == "gaussian":
        dp_count_rdd = add_gaussian_noise_to_rdd(dp_count_rdd, eps, delta, max_partitions, max_per_partition)

    # filter tuples that have less than `min_number_of_views` views
    dp_count_rdd = dp_count_rdd.filter(lambda x: x[1] >= calculate_threshold(delta, eps, max_partitions, max_per_partition))

    # round view count to integers for readability
    dp_count_rdd = dp_count_rdd.map(lambda x: (x[0], round(x[1], 0)))

    dp_count_rdd.takeOrdered(200, key=lambda x: -x[1])

In [9]:
# total contributions (aka sensitivity) = max_per_partition * max_partitions
max_partitions = 5    # say that users can visit at most 5 pages
max_per_partition = 2 # and for each page they can contribute at most 2 pageviews

eps = 1
delta = 5e-8

In [10]:
do_count(rdd, eps, delta, max_partitions, max_per_partition, "laplace")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 3.0 failed 4 times, most recent failure: Lost task 8.3 in stage 3.0 (TID 283, an-worker1135.eqiad.wmnet, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/lib/hadoop/data/d/yarn/local/usercache/htriedman/appcache/application_1629727304304_37996/container_e27_1629727304304_37996_01_000004/pyspark.zip/pyspark/worker.py", line 364, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/var/lib/hadoop/data/d/yarn/local/usercache/htriedman/appcache/application_1629727304304_37996/container_e27_1629727304304_37996_01_000004/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/var/lib/hadoop/data/d/yarn/local/usercache/htriedman/appcache/application_1

In [ ]:
do_count(rdd, eps, delta, max_partitions, max_per_partition, "gaussian")